In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pathlib
import time
import os
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


import pandas as pd

from tqdm import tqdm



In [2]:
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 8.29M/8.29M [00:00<00:00, 42.2MB/s]


In [3]:
path = pathlib.Path('../../data/netflix/netflix_official_.csv')
netflix_csv = pd.read_csv(path)

In [ ]:
imdbid = netflix_csv['imdbid']

browser = webdriver.Chrome(service=service,options=chrome_options)
browser.implicitly_wait(5)
result=pd.DataFrame()

for imdb in tqdm(imdbid):
    url = f"https://www.imdb.com/title/{imdb}/reviews"
    browser.get(url)
    for i in range(10):
        try:
            load = browser.find_element(By.CSS_SELECTOR,'button.ipl-load-more__button')
            load.click()
            time.sleep(0.4)
        except:
            print(i)
            break

    title = browser.find_elements(By.CSS_SELECTOR,'a.title')
    title_list = []
    for t in title:
        # print(t.text)
        # print('-')
        title_list.append(t.text)

    contents = browser.find_elements(By.CSS_SELECTOR,'div.text.show-more__control')
    contents_list = []
    for c in contents:
        # print(c.text)
        # print('-'*50)
        contents_list.append(c.text)
    contents_list=list(filter(None,contents_list))
    df = pd.concat([pd.DataFrame({'title':[title_list]}),pd.DataFrame({'contents':[contents_list]}),pd.DataFrame({'imdbid':[imdb]})]
                    ,axis=1)
    df.index = [netflix_csv[netflix_csv['imdbid']==imdb]['제목'].values[0]]
    result = pd.concat([result,df])
result.to_csv('imdb_review.csv',encoding='utf-8')
        

In [5]:
result

,title,contents,imdbid
우주 히어로 키드,"[so, a kid..., A Show Worth Watching, Craig Mc...",[A show created by Craig McCracken is enough t...,tt9248538
청소년 v 정부: 기후 정의를 외치다,[],[],tt13192640
하프 배드: 선 & 데빌,"[Excellent movie and loved the soundtrack., EX...",[I loved the movie and the soundtrack was exce...,tt16380676
Breaking Bad,"[Really Great, Damn near perfect!, Among the b...",[I have never watched a show that is as consis...,tt0903747
우리의 지구,"[Wow, Done before, But not like this, Make our...",[How was this filmed?? I wish they make a docu...,tt9253866
...,...,...,...
해피 데스데이,"[Surprisingly better than I expected!, 'Ground...",[This type of movie has been done many times b...,tt5308322
임금님의 사건수첩,[The king of Joseon is now a Sherlock Holmes-s...,"[Loving detective movies, this Korean movie ha...",tt6817202
아이 엠 브렌트 모린,[A perspective after all the hype has died dow...,[Lucas may have problems as a director and wri...,tt0120915
알카티브,[Lame? Waste of Talent? Bollocks! This is terr...,"[Sorry to whomever thought this was ""a lame wa...",tt0119632
